In [8]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import torch
import numpy as np
import json
import os
import cv2
import imutils
import random
model = YOLO('best.pt')

In [15]:
def rounding_up(matrix): # заменяет все числа на 1 и 0
    """
    массив, что содержит в себе все цвета каждого пикселя преобразует числа цветов в 1 и нолики (0 - чёрный цвет, а 1 - белый цвет)
    """
    result = np.zeros_like(matrix)
    result[matrix > 150] = 1
    return result

def split_matrix(matrix, rows, cols):
    """
    разделяет список на 49 частей (то есть разделяет весь массив, картинку на матрицу 7 на 7 клеток), после избавляется от крайних столбцов и строк => выводит матрицу 5 на 5
    """
    row_size, col_size = rows // 7, cols // 7
    parts = []
    for i in range(7):
        for j in range(7):
            part = matrix[i * row_size : (i + 1) * row_size, j * col_size : (j + 1) * col_size]
            parts.append(part)

    central_parts = []
    for i in range(1, 6):
        for j in range(1, 6):
            central_parts.append(parts[i * 7 + j])
    
    return central_parts

def replace_based_on_center(parts):
    """
    округляет каждую часть списка до 1 или 0 на основе цвета центрального пикселя
    """
    replaced_parts = []
    for part in parts:
        center_pixel = part[len(part) // 2, len(part[0]) // 2]
        replaced_part = np.full_like(part, center_pixel)
        replaced_parts.append(replaced_part)
    
    return replaced_parts

def combine_parts(rounded_parts):
    """
    сохраняет всё в виде матрицы 5 на 5, где каждая клетка равна либо положительному числу, либо нулю
    """
    combined_matrix = np.zeros((5, 5))
    for i in range(5):
        for j in range(5):
            part_index = i * 5 + j
            combined_matrix[i, j] = 1 if np.sum(rounded_parts[part_index]) > 0 else 0
    return combined_matrix

def read_from_json(filename):
    # Открытие файла для чтения
    with open(filename, 'r') as json_file:
        # Загрузка данных из файла
        data = json.load(json_file)
    return data

def load_json_to_matrices(filename):
    """
    загрузка базы данных в список matrices и преобразование его в словарь numpy
    """
    with open(filename, 'r') as json_file:
        data = json.load(json_file)

    matrices = {}
    for key, value in data.items():
        matrices[key] = np.array(value)
    
    return matrices

def compare_matrices(matrix1, matrices_from_json):
    """
    сравнение нашей матрицы с матрицами из базы данных
    """
    for key, matrix2 in matrices_from_json.items():
        if np.array_equal(matrix1, matrix2):
            return key
    return None

def image_optimization(image):
    """
    Изменяет качество картинки до изображение в 70 на 70 пикселей
    """
    enlarged_image = cv2.resize(image, (70, 70), interpolation=cv2.INTER_CUBIC)
    optimized_image_path = 'optimized_image.png'
    cv2.imwrite(optimized_image_path, enlarged_image)
    return optimized_image_path

def main(picture, json_file): 
    """
    Финальная функция, что содержит в себе все предыдущие. main возвращает номер карточки и вариант её ответа, если же данная картачка не
    совпадает ни с одной из карточек из базы данных, то возвращается None. (Принимает на вход саму картинку и файл с базой данных)
    """
    cb_img = cv2.imread(picture, 0)
    cb_img_np = np.array(cb_img)
    lines, columns = cb_img.shape
    the_converted_image = combine_parts(replace_based_on_center(split_matrix(rounding_up(cb_img_np), lines, columns)))
    return compare_matrices(the_converted_image, load_json_to_matrices(json_file))
    # Предполагаем, что cb_img_np уже является массивом numpy.ndarray, представляющим изображение в оттенках серого


def mainer(cb_img_np, json_file):
    """
    Финальная функция, что содержит в себе все предыдущие. main возвращает номер карточки и вариант её ответа, если же данная картачка не
    совпадает ни с одной из карточек из базы данных, то возвращается None. (Принимает на вход массив numpy.ndarray и файл с базой данных)
    """

    cb_img_np = cv2.cvtColor(cb_img_np, cv2.COLOR_BGR2GRAY)
    cb_img=image_optimization(cb_img_np)
    cb_img = cv2.imread(cb_img, 0)
    cb_img_np = np.array(cb_img)
    lines, columns = cb_img_np.shape[:2]
    the_converted_image = combine_parts(replace_based_on_center(split_matrix(rounding_up(cb_img_np), lines, columns)))
    return compare_matrices(the_converted_image, load_json_to_matrices(json_file))


In [4]:
#Трансформация карточек (Выравние)
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):

    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [5]:
#функция вырезки карточек из изображения
def crop_images(img, coordinates):
    images = []
    for coords in coordinates:
        left, top, right, bottom = map(int, coords)
        cropped_img = img[top-5:bottom+5, left-5:right+5]
        images.append(cropped_img)
    return images

In [30]:
#----------------------------------------------------------------

In [6]:
#Получение изображения с карточками в ограничивающей рамке
def get_processed_photo(photo):
    photo=cv2.imread(photo)
    results=model(photo,conf=0.8)
    im_array = results[0].plot()
    return im_array
result_photo=get_processed_photo('fKPZ3p7CjU8.jpg')
cv2.imshow('result',result_photo)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x480 (no detections), 300.0ms
Speed: 5.9ms preprocess, 300.0ms inference, 1421.2ms postprocess per image at shape (1, 3, 640, 480)


In [29]:
#Сохранение вырезанных карточек без ориентации
def save_crop_photos(img,save_dir):
    results=model(img,conf=0.8)
    results[0].save_crop(save_dir)
save_crop_photos(cv2.imread('100.jpg'),'C:/Users/Admin/Downloads/dfdsf/save_img')

In [11]:
#Сохранение выраезанных карточек с ориентацией
def save_image_with_sequence_number(image, directory, base_name):
    # Проверка существования директории, и если её нет, то создание
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Получение списка файлов в директории
    files = os.listdir(directory)
    
    # Вычисление номера для нового файла
    number = len(files) + 1
    
    # Составление имени файла с учетом базового имени и номера
    file_name = os.path.join(directory, f"{base_name}{number}.jpg")
    
    # Сохранение изображения
    cv2.imwrite(file_name, image)
    
    print(f"Изображение сохранено как {file_name}")


In [12]:
#отрисовка рамок с подписями ответов
def draw_bounding_box(image, box, card_name):
    #color = (random.randint(70, 255), random.randint(70, 255), random.randint(70, 255))
    color=(0,255,0)
    box=[int(num) for num in box]
    start_x, start_y, end_x, end_y = box
    cv2.rectangle(image, (start_x, start_y), (end_x, end_y), color, 4)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottom_left_corner_of_text = (10, 30)
    font_scale = 1
    line_type = 0
    thickness = 5
    cv2.putText(image, card_name, (start_x,start_y-6), font, font_scale, color,thickness)
    #cv2.imshow("Image with Bounding Box", image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return image


In [13]:
#Поиск контура карточки
def find_largest_contour(image):
    # Переводим изображение в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Применяем пороговую обработку для получения бинарного изображения
    _, thresh = cv2.threshold(gray, 127, 255, 0)
    # Инвертируем бинарное изображение
    thresh = cv2.bitwise_not(thresh)
    # Находим контуры на инвертированном бинарном изображении
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [17]:
#Получение результата по фото(Новый)
from copy import deepcopy
def get_results_photo2(img):
    ans=None
    answers = []
    if img is None:
        print("Error: The image is empty or not loaded correctly.")
        return answers, img
    copy_img=deepcopy(img)
    results = model(img, verbose=False,conf=0.7)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    images = crop_images(img, boxes)
    for i in range(len(images)):
        image = images[i]
        if image is None or image.size == 0:
            print(f"Error: The cropped image at index {i} is empty.")
            continue
        orig = image
        image = cv2.GaussianBlur(image, (5,5), 0)
        
        edged = cv2.Canny(image, 0,0)
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        
        cnts+= find_largest_contour(image)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        
        screenCnt = None
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.1 * peri, True)
            if len(approx) == 4 and cv2.contourArea(approx)>=20:
                screenCnt = approx
                if screenCnt is not None:
                    warped = four_point_transform(orig, screenCnt.reshape(4, 2))
                    try:
                        ans=mainer(warped, 'image_database.json')
                        draw_bounding_box(copy_img, boxes[i], ans)
                    except:
                        pass
                    if ans not in answers and ans!=None:
                        answers.append(ans)
                        
    return answers, copy_img
model = YOLO('best.pt')
answers,img=get_results_photo2(cv2.imread("HAUvT7Whd84.jpg"))#Получение итогового изображения и ответов
print(answers)
height, width = img.shape[:2]
    
    # Уменьшите изображение в 2 раза
img = cv2.resize(img, (width // 2, height // 2), interpolation=cv2.INTER_AREA)
cv2.imshow("Result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

['4A']


In [19]:
#Получение результата по фото(Старый)
def get_results_photo1(img):
    answers = []
    if img is None:
        print("Error: The image is empty or not loaded correctly.")
        return answers, img
    results = model(img, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    images = crop_images(img, boxes)
    for i in range(len(images)):
        image = images[i]
        if image is None or image.size == 0:
            print(f"Error: The cropped image at index {i} is empty.")
            continue
        orig = image
        image = cv2.GaussianBlur(image, (5, 5), 0)
        
        edged = cv2.Canny(image, 120,255)
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        
        cnts+= find_largest_contour(image)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        
        screenCnt = None
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.04 * peri, True)
            if len(approx) == 4:
                screenCnt = approx
                if screenCnt is not None:
                    warped = four_point_transform(orig, screenCnt.reshape(4, 2))
                    try:
                        ans=mainer(warped, 'image_database.json')
                    except:pass
                    if ans not in answers:
                        answers.append(ans)
    return answers
answers=get_results_photo1(cv2.imread("0vUota9j3n0.jpg"))
print(answers)

[None, '32B', '36B', '31C', '39B', '40C', '35C', '2A', '1C', '38D', '37C']


In [13]:
#Функция для нахождения карточек на больших фотографиях (Разделение фото на более мелкие и их прогон через нейросеть)
import cv2
from patched_yolo_infer import MakeCropsDetectThem, CombineDetections
def find_hard_cards(img):
    element_crops = MakeCropsDetectThem(
        image=img,
        model_path='best.pt',
        segment=False,
        shape_x=640,
        shape_y=640,
        overlap_x=10,
        overlap_y=20,
        conf=0.3,
        iou=0.7,
        resize_initial_size=True,
    )
    result = CombineDetections(element_crops, nms_threshold=0.25, match_metric='IOS')  
    boxes=result.filtered_boxes
    return boxes

In [39]:
#Функция для обработки и сохранения видео
import cv2
def extract_frames(video_path):
    # Загрузка видео
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    # Получение параметров видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    size = (frame_width, frame_height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Используйте 'mp4v' для MP4
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Здесь можно вызвать вашу функцию обработки кадра
        _,processed_frame = get_results_photo2(frame)
        frames.append(processed_frame)
    
    cap.release()
    return frames, fps, size, fourcc

def save_video(frames, output_path, fps, size, fourcc):
    # Создание объекта VideoWriter
    out = cv2.VideoWriter(output_path, fourcc, fps, size)
    
    for frame in frames:
        out.write(frame)
    
    out.release()

# Пример использования
video_path = 'video_20240605_150952.mp4'
frames, fps, size, fourcc = extract_frames(video_path)
output_path = '9999.mp4'
save_video(frames, output_path, fps, size, fourcc)

In [13]:
#демонстрация работы модели на видео
def real_time_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            results = model(frame,verbose=False,conf=0.8)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
    cap.release()
    cv2.destroyAllWindows()

real_time_video('video_20240605_150952.mp4')

In [21]:
#демонстрация работы модели + алгоритма распознования карточек на видео 
def video_detect_card(video_path):
    frame_counter=0
    cap = cv2.VideoCapture(video_path)
    answers = set()
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            frame_counter += 1
            # Display the annotated frame
            result,img = get_results_photo2(frame)
            cv2.imshow("YOLOv8 Inference", img)
            for value in result:
                    answers.add(value)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
    cap.release()
    cv2.destroyAllWindows()
    return(answers)

print(video_detect_card('video_20240605_150952.mp4'))

set()



0: 640x640 17 cards, 45.0ms
Speed: 784.0ms preprocess, 45.0ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)


In [22]:
#Демонстрация работы нейросети на видео в реальном времени
model = YOLO('best.pt')
def Real_time_demonstration():
    # Захватываем видеопоток с первой доступной веб-камеры
    cap = cv2.VideoCapture(0)

    while True:
        # Считываем кадр с веб-камеры                     
        ret, frame = cap.read()
        results = model(frame,verbose=False,conf=0.8)
        cv2.imshow('Webcam', results[0].plot())
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Освобождаем ресурсы
    cap.release()
    cv2.destroyAllWindows()

# Запускаем функцию для вывода видео с веб-камеры
Real_time_demonstration()

In [20]:
#Демонстрация работы нейросети + алгоритма распознования карточек в реальном времени Надо переделать (завтра)
def Real_time_define():
    # Захватываем видеопоток с первой доступной веб-камеры
    cap = cv2.VideoCapture(0)
    frame_counter=0
    answers=set()
    while True:
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            _,img=get_results_photo2(frame)
            cv2.imshow("YOLOv8 Inference", img)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Освобождаем ресурсы
    cap.release()
    cv2.destroyAllWindows()
    return answers
print(Real_time_define())

Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 1 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 2 is empty.
Error: The cropped image at index 3 is empty.
set()
